In [2]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parsear XML
tree = ET.parse("../results/report.xml")
root = tree.getroot()

# Estructura: <results><errors><error>...
rows = []

for error in root.findall(".//error"):
    err_id = error.attrib.get("id")
    severity = error.attrib.get("severity")
    msg = error.attrib.get("msg")
    verbose = error.attrib.get("verbose")
    cwe = error.attrib.get("cwe", "")
    inconclusive = error.attrib.get("inconclusive", "false")
    symbol = error.findtext("symbol")

    # Puede haber múltiples ubicaciones por error
    for loc in error.findall("location"):
        file = loc.attrib.get("file")
        line = loc.attrib.get("line")
        col = loc.attrib.get("column")
        rows.append({
            "ID": err_id,
            "Severity": severity,
            "Symbol": symbol,
            "Message": msg,
            "Verbose": verbose,
            "CWE": cwe,
            "Inconclusive": inconclusive,
            "File": file,
            "Line": line,
            "Column": col
        })

# Crear DataFrame
df_cppcheck = pd.DataFrame(rows)
df_cppcheck

,ID,Severity,Symbol,Message,Verbose,CWE,Inconclusive,File,Line,Column
0,functionStatic,performance,Worker::process,Technically the member function 'Worker::proce...,The member function 'Worker::process' can be m...,398,true,../../../apps/calcjob/Worker.hpp,55,10
1,functionStatic,performance,Reporter::opToStr,Technically the member function 'Reporter::opT...,The member function 'Reporter::opToStr' can be...,398,true,../../../apps/calcjob/Reporter.hpp,14,17


In [5]:
with open("../results/cppcheck_report.md", "w") as f:
    f.write("## Issues detectados por Cppcheck\n\n")
    f.write(df_cppcheck.to_markdown(index=False))